In [ ]:
%pip install -qU openai marvin
%pip install -qU "psycopg[binary]"
%pip install tiktoken

## Getting ready

Begin by:
1. creating a new directory `data/<jurisdiction>` and populate with one or more
docx files containing the jurisdiction's municipal code
2. run `scripts/convert_docx.sh` to convert those files into a single text file
3. make a copy of `notebooks/template-workflow.ipynb` to `notebooks/<jurisdiction>.ipynb`
and continue processing in that notebook

In [2]:
import sys
sys.path.insert(0, '..')

## set up auto-reloading for development
%reload_ext autoreload
%autoreload 2

## Specify heading patterns

Replace the `jurisdiction_headings` dict with examples from your jurisdiction

In [3]:
from muni.code import Jurisdiction

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
  input="It was the best of times, it was the worst of times.",
  model="text-embedding-3-small"
)

print(response.data[0].embedding)

In [5]:
heading_examples = {
    1: ["TITLE 1\nGENERAL PROVISION\n",
        "TITLE 2\nCITY GOVERNMENT AND ADMINISTRATION\n",
        "TITLE 3\nREVENUE AND FINANCE\n",
        ],
    2: ["CHAPTER 1-4\nCODE ADOPTION - ORGANIZATION\n",
        "CHAPTER 1-8\nCITY SEAL AND FLAG\n",
        "CHAPTER 1-12\nCITY EMBLEMS\n",
        ],
    3: ["1-4-010 Municipal Code of Chicago adopted.\n",
        "2-1-020 Code to be kept up-to-date.\n",
        "3-4-030 Official copy on file.\n",
        ],
}

In [6]:
from muni.code import infer_heading_patterns, infer_level_names

In [ ]:
## Verify that the regular expressions matching outline levels look okay
heading_patterns = infer_heading_patterns(heading_examples)
for level, pattern in heading_patterns.items():
    print(f"{level}: r'{pattern.regex}'")

print()

## Verify that the names of the sections look okay
level_names = infer_level_names(heading_patterns)
for level, name in level_names.items():
    print(f"{level}: {name}")

## Specify the parameters of the jurisdiction and parse the code

In [8]:
place = Jurisdiction(
    name="Chicago Mini",
    title="Municipal Code of Chicago",
    patterns=heading_patterns,
    level_names=level_names,
    source_local="../data/chicago-mini/code.txt",
    source_url="https://www.chicago.gov/city/en/depts/doit/supp_info/municipal_code.html",
)

place.parse()
place.chunkify(1000)

In [ ]:
## Verify that the distribution of paragraphs and chunks looks okay
place.summarize()

## Upload data to the database

In [10]:
from muni.code import upload

db = {'dbname': 'muni',
      'user': 'muni',
      'password': '',
      'host': 'localhost',
      'port': 5432}

upload(db, place)

In [11]:
from muni.code import upload_embeddings, refresh_views

upload_embeddings(db, place)
refresh_views(db)

## Find associations among sections

In [12]:
from muni.code import find_associations

find_associations(db, place)
# TODO: changing DB schema

## Basic queries

In [ ]:
from muni.code import simple_semantic_query

queries = ['Does the municipal code contain provisions restricting the use of drug paraphernalia?']

for query in queries:
    print(f"Query: {query}")
    results = simple_semantic_query(db, place, query, limit=20)
    for result in results:
        print(result)

In [ ]:
from muni.code import extract_keywords, simple_full_text_query

## FIXME: this doesn't work well because extract_keywords() returns too many keywords
# queries = [' '.join(extract_keywords(query)) for query in queries]

queries = ['drug paraphernalia']

for query in queries:
    print(f"Query: {query}")
    results = simple_full_text_query(db, place, query, limit=20)
    for result in results:
        print(result)


In [ ]:
from muni.code import hybrid_query

queries = ['Does the municipal code contain provisions restricting the use of drug paraphernalia?']

for query in queries:
    print(f"Query: {query}")
    results = hybrid_query(db, place, query, limit=20)
    for result in results:
        print(result)

## Report generation

In [ ]:
from muni.code import Report
from IPython.display import display, Markdown

query = 'Does the municipal code contain provisions restricting the use of drug paraphernalia?'

report = Report(db, place, query)

display(Markdown(str(report)))

## Upload results to database

In [17]:
from muni.code import upload_report

upload_report(db, report)

# Evaluation

In [ ]:
queries = [ ] #Set of queries seperated by comma (,)

gt_labels = [ ] #Ground Truth Labels - 0 for No and 1 for Yes

# Qualititve Analysis

This analysis measures model performance using objective metrics such as accuracy. It includes comparing generated outputs against ground truth labels(gt_lables) to evaluate correctness and reliability at scale.

#### Muniscope

In [ ]:
city_name   = "CITY NAME"          # <-- pick the city you want
juris       = place         # the Jurisdiction obj you already built
gt_labels   = gt_labels # <-- 0=no, 1=yes for each query

# The nine DPL priority questions in order:
queries = queries

def yn_to_int(ans):
    """
    yes  → 1
    no   → 0
    None → 0   # <- treat missing answer as 'no'
    """
    if ans is None:
        return 0
    return 1 if ans.strip().lower().startswith("y") else 0


# ------------------------------------------------------------
#   RUN
# NA - for no answer?
#  
# ------------------------------------------------------------

print(f"Results for {city_name}:\n")
correct = 0
for i, q in enumerate(queries):
    report = Report(db, juris, q)          # runs retrieval + QA
    pred   = yn_to_int(report.short_answer)       # now never raises
    truth  = gt_labels[i]

    if report.short_answer is None:
        disp = "—"            # show dash for “no answer”
    else:
        disp = report.short_answer

    if pred == truth:
        correct += 1
    print(f"{i+1}. Model: {disp}  |  Truth: {truth}  "
          f"{'✓' if pred==truth else '✗'}")

accuracy = correct / len(queries)
print(f"\nAccuracy: {accuracy:.2%}  ({correct}/{len(queries)})")

#### Gpt-4o

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

client = OpenAI()  

queries_x = [ ] 
"""Use a set of queries, but include the city’s name in each query to ensure GPT-4o interprets them correctly.
Example:
For Chicago_IL: ‘Are there exemptions specifically for fentanyl testing or checking equipment?"""

gt_labels   = gt_labels

def ask_openai(prompt: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Answer only with 'Yes' or 'No'."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    return response.choices[0].message.content.strip()

correct = 0
predictions = []

for i, query in enumerate(queries):
    answer = ask_openai(query).strip().lower()

    if answer == "yes.":
        pred_label = 1
    elif answer == "no.":
        pred_label = 0
    else:
        pred_label = -1  # Invalid response

    predictions.append(pred_label)

    gt_label = gt_labels[i]
    is_correct = pred_label == gt_label

    # Print result
    print(f"Q{i+1}: {query}")
    print(f"→ Model: {answer} | GT: {'Yes' if gt_label else 'No'} | "
          f"{'✅' if is_correct else ('❌ (Invalid)' if pred_label == -1 else '❌')}")
    print()

    if is_correct:
        correct += 1

accuracy = correct / len(queries)
print(f"Accuracy: {accuracy:.2%}")

# Qualititive Analysis

This analysis involves manually reviewing the model's responses to assess clarity, accuracy, and alignment with legal context. It helps identify nuanced issues like misinterpretation of city-specific terms or inconsistent language use.

In [ ]:
# Muniscope

queries = queries
# Loop through each query and display its report
for i, query in enumerate(queries, 1):
    print(f"\n{'='*80}\nQuery {i}: {query}\n")
    try:
        report = Report(db, place, query)
        display(Markdown(str(report)))
    except Exception as e:
        print(f"⚠️ Failed to generate report for Query {i}: {e}")

In [ ]:
# GPT-4o

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI() 

queries = queries_x

def ask_openai(prompt: str) -> str:
    system_prompt = (
        "You are a legal analysis assistant. For each query, return a detailed answer citing the relevant municipal ordinance, "
        "section, or paragraph if possible. If no reference is found, state that clearly. Be precise and quote text from the code if applicable."
    )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    return response.choices[0].message.content.strip()

for i, query in enumerate(queries):
    answer = ask_openai(query)

    print(f"Q{i+1}: {query}")
    print(f"→ Detailed Answer with Reference: {answer}")
    print()
